
# Projet: Facebook BaBi tasks
Notebook de la semaine du 27/02
Par Thierry Loesch et Bryce TIchit

Sur ce notebook nous avons rajouté un parser afin de parser les données et nous avons fait la vectorisation de celles-ci. Un modèle a aussi été créé selon ce qui nous avions défini dans le notebook précédent. Nous avons ainsi pu commencer a faire des tests avec le modèle, et nous avons eu des résultats plutôt satisfaisant sur la première task, environ 70% de précision. Le réseau récurrent utilisé est LSTM sur ce notebook.

Reste à perfectionner le modèle et/ou les autres paramètres. Essayer les autres tasks également. Utiliser un Memory Network pour après?


In [23]:
import re

def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parserBabi(data):
    ret=list()
    story=list()
    
    for phrase in data:
        phrase = phrase.decode('utf-8').strip()
        id_phrase,phrase = int(phrase[0]),phrase[1:]
        
        if id_phrase == 1: #Nouvelle story
            story=list()
            
        if '\t' in phrase: #Si tabulation alors il s'agit de la question ainsi que de la réponse
            q, a, justif = phrase.split('\t')
            q = tokenize(q)
            
            data_story = [x for x in story if x] 
                                             
            ret.append((data_story,q,a)) #Nos données d'apprentissages
            story.append('')
        
        else: 
            #Alors la phrase est tout simplement un des élements de raisonnement et non une question
            story.append(tokenize(phrase))
            
    return ret

def readAndParse(f):
    data = parserBabi(f.readlines())
    return [([substory for substories in story for substory in substories], q , a) for story,q,a in data]


# Récupération des données

On récupère les fichiers sur internet directement avec la fonction get_file, cette fonction a l'avantage de ne pas tout retelecharger si les données sont déjà sur la machines (dans ~/.keras/datasets). Puis on applique les fonctions du parser afin de récupérer les données dans la structure que l'on veut.

In [24]:
from keras.utils.data_utils import get_file
import tarfile

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz')
except:
   print("erreur pendant le telechargement")
    
tar = tarfile.open(path)
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
#challenge = 'tasks_1-20_v1-2/en/qa19_path-finding_{}.txt'

train = readAndParse(tar.extractfile(challenge.format('train')))
test = readAndParse(tar.extractfile(challenge.format('test')))

vocab = sorted(reduce(lambda x, y: x | y, (set(story + q + [answer]) for story, q, answer in train + test)))
vocab_size = len(vocab) + 1
word_idx = dict(((w,i+1) for i,w in enumerate(vocab)))

story_max = max((len(x) for x,_,_ in train+test))
question_max = max((len(x) for _,x,_ in train+test))


# Vectorisation

Comme vu en cours on doit vectoriser nos données (les assimiler a des nombres et les mettre dans des matrices) afin de pouvoir entrainer notre réseau neuronal.

La fonction pad_sequences plus bas permet de transformer notre liste de liste en matrice numpy en ajoutant des 0. pour complèter quand il n'y a pas de données.

In [26]:
#Mettons tout ça dans des matrices
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def vectorize(data,word_idx,story_max,question_max):
    
    X = []
    Xq = []
    Y = []
    for story, question, reponse in train:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in question]
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[reponse]] = 1 
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    
    return pad_sequences(X, maxlen=story_max), pad_sequences(Xq, maxlen=question_max), np.array(Y)

X,Xq,Y = vectorize(train,word_idx,story_max,question_max)
X_test,Xq_test,Y_test = vectorize(test,word_idx,story_max,question_max)

In [27]:
print X[0:10]
print Xq[0:10]
print Y[0:10]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  7 18 21 20 11  1  6 23 21 20 14  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7 18 21 20 11  1  6 23 21
  20 14  1  5 23 10 21 20 14  1  8 18 21 20 13  1]
 [ 0  0  0  7 18 21 20 11  1  6 23 21 20 14  1  5 23 10 21 20 14  1  8 18
  21 20 13  1  6 18 21 20 19  1  8 16 21 20 11  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  8 22 21 20 19  1  8 23 21 20 11  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8 22 21 20 19  1  8 23
  21 20 11  1  7 23 21 20 12  1  5 18 21 20 14  1]
 [ 0  0  0  0  8 22 21 20 19  1  8 23 21 20 11  1  7 23 21 20 12  1  5 18
  21 20 14  1  6 23 21 20 13  1  6 22 21 20 19  1]


# Modèle

Ensuite nous crééons notre modèle keras, nous avions choisi donc de faire un modèle pour les story et pour les questions afin de pouvoir raisonner sur chacun d'eux différement puis de les combiner en un modèle, ainsi nous avons un seul modèle mais qui traite différement les story des questions. On a un LSTM dans le modèle des story car il faut raisonner d'abord sur les story seules puis les story avec les questions (raisonner sur les questions seule ne veut pas dire grand chose)

Schema modèle:

                    story_model                              question_model
                         |                                           |
                      Embedding                                  Embedding
                          |                                          |
                                                                    /
                        LSTM                                       /
                          |                                       /
                        RepeatVector                            /
                                    \                          /
                                      \_______________________/
                                                   |
                                                   |
                                              Merge(mode=sum)
                                                   |
                                                 LSTM
                                                   |
                                                 SoftMax

Notes:

- Avec un batchsize de 32 un nombre de 60 epochs semble optimal.
- Le fait d'avoir utilisé un batchsize petit a amelioré les résultats (32 optimal?)


In [21]:
#Model

from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout,RepeatVector,Activation
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop,Adam

embed_size = 50
batch_size=32
epochs=60

story_model = Sequential()
story_model.add(Embedding(vocab_size,embed_size,input_length=story_max))


question_model = Sequential()
question_model.add(Embedding(vocab_size,embed_size,input_length=question_max))
question_model.add(LSTM(embed_size))
question_model.add(RepeatVector(story_max)) #permet d'ajuster la taille du modèle afin de préparer un merge

model = Sequential()
model.add(Merge([story_model, question_model], mode='sum'))
model.add(LSTM(embed_size))
model.add(Dense(vocab_size))
model.add(Activation("softmax"))

model.compile(optimizer=RMSprop(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit([X, Xq], Y, batch_size=batch_size, nb_epoch=epochs)


Epoch 1/60
1000/1000 [==============================] - 2s - loss: 1.9705 - acc: 0.1440     
Epoch 2/60
1000/1000 [==============================] - 2s - loss: 1.7229 - acc: 0.2970     
Epoch 3/60
1000/1000 [==============================] - 2s - loss: 1.5753 - acc: 0.4070     
Epoch 4/60
1000/1000 [==============================] - 2s - loss: 1.4939 - acc: 0.4050     
Epoch 5/60
1000/1000 [==============================] - 2s - loss: 1.4186 - acc: 0.4270     
Epoch 6/60
1000/1000 [==============================] - 2s - loss: 1.3726 - acc: 0.4190     
Epoch 7/60
1000/1000 [==============================] - 2s - loss: 1.3177 - acc: 0.4250     
Epoch 8/60
1000/1000 [==============================] - 2s - loss: 1.3046 - acc: 0.4310     
Epoch 9/60
1000/1000 [==============================] - 2s - loss: 1.2544 - acc: 0.4300     
Epoch 10/60
1000/1000 [==============================] - 2s - loss: 1.2060 - acc: 0.4600     
Epoch 11/60
1000/1000 [==============================] - 2s - loss: 1

In [22]:
#Calcul de précision sur l'ensemble de test

loss,acc = model.evaluate([X_test, Xq_test],Y_test, batch_size=batch_size)
print "\nPerte = {}".format(loss)
print "Précision = {}".format(acc)

def getClass(nd_matrix):
    return np.argmax(nd_matrix,axis=1)
    

 992/1000 [============================>.] - ETA: 0s
Perte = 0.712211671829
Précision = 0.688
